In [1]:
import numpy as np

In [2]:
def pprint_dict(in_dict):
    import simplejson as json
    print(json.dumps(in_dict,  indent=2))

def next_player(p):
    p += 1
    p %= 2
    return p

In [3]:
# DEPRECATED WITH NEW VERSION OF ENCODING TO NUMPY ARRAYS
def serialize_state(grid):
    flat_grid = sum(grid.astype(str).tolist(), [])
    flat_grid_str = ''.join(flat_grid)
    return flat_grid_str


def unserialize_state(grid_str):
    return np.array([list(grid_str)]).reshape(8, 8).astype(int)

In [4]:
def find_legal_moves(grid, p):
    assert p in (0, 1), "player not valid"

    def find_legal_moves_row(grid):
        moves = []
        for i in range(len(grid)):
            for j in range(len(grid.T)-1):
                if grid[i, j] == 0 and grid[i, j+1] == 0:
                    grid_next = grid.copy()
                    grid_next[i, j] = 1
                    grid_next[i, j+1] = 1
                    moves.append(grid_next)
        return moves

    def find_legal_moves_col(grid):
        moves = []
        for i in range(len(grid)-1):
            for j in range(len(grid.T)):
                if grid[i, j] == 0 and grid[i+1, j] == 0:
                    grid_next = grid.copy()
                    grid_next[i, j] = 1
                    grid_next[i+1, j] = 1
                    moves.append(grid_next)
        return moves

    if p :
        return find_legal_moves_col(grid)
    else:
        return find_legal_moves_row(grid)


def choose_next(grid, p, decision='random'):
    from random import choice
    if decision == 'random':
        grid_next_all = find_legal_moves(grid, p)
        if len(grid_next_all) > 0:
            return choice(grid_next_all)
    return


def playout(grid, p):
    """
    Perform random play until end
    returns True if p wins , False otherwise
    """
    p_0 = p

    sanity_counter = 64
    while sanity_counter > 0:
        sanity_counter -= 1
        
        grid = choose_next(grid, p)
        
        if grid is not None:
            p = next_player(p)
        else:
            return p_0 != p
    print("Sanity counter expired; problem with playout")




def eval_next_moves(grid_ini, p_ini, n_playouts=20):
    """
    For each possible move play {n_playouts=100} playouts and
    calculate the average of the playouts
    """

    legal_moves = find_legal_moves(grid_ini, p_ini)
    if len(legal_moves) == 0:
        return

    result = {}
    for m in legal_moves:
        win_seq_bool = []
        for _ in range(n_playouts):
            win_seq_bool.append(playout(m, p_ini))
        elo =  sum(win_seq_bool) / len(win_seq_bool)
        result[serialize_state(m)] = elo
    
    print("Best move, player: {}".format(p_ini))
    return result


def mt_game(grid_ini, p_ini):
    grid = grid_ini.copy()
    p = p_ini
    while True:
        res = eval_next_moves(grid, p)
        if res is None:
            print("PLAYER {p} WINS".format(p=next_player(p)))
            return
        grid = unserialize_state(max(res.items(), key = lambda x : x[1])[0])
        print(grid)
        print()
        p = next_player(p)


def find_best(grid_ini, p_ini):
    res = eval_next_moves(grid, p)
    if res is None:
        return
    return unserialize_state(max(res.items(), key = lambda x : x[1])[0])

if __name__ == "__main__":

	# Generate games using a Monte Carlo evaluation to choose moves at Domineering.

	p = 0
	grid_current = np.array( [ [0] * 8 ] * 8 )
	#print(grid_current)
	print()

	mt_game(grid_current, 0)



Best move, player: 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Best move, player: 1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Best move, player: 0
[[0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Best move, player: 1
[[0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Best move, player: 0
[[0 0 0 1 1 1 1 0]
 [0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

Best move, player: 1
[[0 0 0 1 1 1 1 0]
 [0 1 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 